In [ ]:
import pandas as pd

In [ ]:
df= pd.read_csv('/content/data-NSE_FO_35000.csv')

In [ ]:
df.head()

,Unnamed: 0,Date,Open,High,Low,Close,Volume,OI
0,1,2024-06-28 03:45:00,24349.90,24403.85,24349.9,24403.85,450,50
1,2,2024-06-28 03:46:00,24399.05,24406.75,24389.7,24400.00,1250,50
2,3,2024-06-28 03:47:00,24400.00,24410.25,24392.0,24400.00,775,50
3,4,2024-06-28 03:48:00,24400.60,24405.00,24400.0,24405.00,275,2050
4,5,2024-06-28 03:49:00,24407.50,24424.90,24400.6,24424.90,2425,2050


In [ ]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
# Convert the 'Date' column to datetime format (optional but recommended)
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
# Localize to UTC, then convert to +5:30
df['Date'] = df['Date'].dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata')

In [ ]:
# Set the 'Date' column as the index
df.set_index('Date', inplace=True)

In [ ]:
df.head()

,Open,High,Low,Close,Volume,OI
Date,,,,,,
2024-06-28 09:15:00+05:30,24349.90,24403.85,24349.9,24403.85,450,50
2024-06-28 09:16:00+05:30,24399.05,24406.75,24389.7,24400.00,1250,50
2024-06-28 09:17:00+05:30,24400.00,24410.25,24392.0,24400.00,775,50
2024-06-28 09:18:00+05:30,24400.60,24405.00,24400.0,24405.00,275,2050
2024-06-28 09:19:00+05:30,24407.50,24424.90,24400.6,24424.90,2425,2050


In [ ]:
pip install backtesting

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173913 sha256=aabf86f95212ab0dd2840f42308b5763a253b01efd170d19a7a982ddb09f2b0e
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [ ]:
def closing(df):
  return df.Close

In [ ]:
from backtesting import Strategy
from backtesting.lib import crossover


class Alphabots(Strategy):
    delta = 0.002
    base_price = df['Close'][0]
    def init(self):
      self.base_price = self.data.Close[0]
      self.delta = 0.002
      self.no_of_trades = 2
      self.short_trade = 0
      self.long_trade = 0
      self.close_prices = self.I(closing, self.data)
      self.square_off = self.data.Close[374]

    def next(self):
      if (self.close_prices == self.square_off) or (self.long_trade+self.short_trade==10):
        self.position.close()
      if self.close_prices > self.base_price*(1+self.delta):
        if self.long_trade <= 5:
          self.buy()
          self.long_trade += 1
      if self.close_prices < self.base_price*(1-self.delta):
        if self.short_trade <= 5:
          self.sell()
          self.short_trade += 1



<ipython-input-63-7fe8037db106>:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  base_price = df['Close'][0]


In [ ]:
from backtesting import Backtest

bt = Backtest(df[:375], Alphabots, cash=100_000, commission=0.00)
stats = bt.run()


In [ ]:
stats

,0
Start,2024-06-28 09:15:00+05:30
End,2024-06-28 15:29:00+05:30
Duration,0 days 06:14:00
Exposure Time [%],92.8
Equity Final [$],99572.8
Equity Peak [$],100028.0
Return [%],-0.4272
Buy & Hold Return [%],-0.165753
Return (Ann.) [%],0.0
Volatility (Ann.) [%],NaN


In [ ]:
stats, heatmap = bt.optimize(base_price=df['Close'][:120].tolist(),
                    delta=[i/1000 for i in range(10, 51)],
                    maximize='Return [%]', max_tries=300,
                        random_state=0,
                        return_heatmap=True)

/usr/local/lib/python3.10/dist-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 305 configurations.
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/backtesting/backtesting.py:1372: PerformanceWarning: indexing past lexsort depth may impact performance.
  heatmap[tuple(params.values())] = value


In [ ]:
stats

,0
Start,2024-06-28 09:15:00+05:30
End,2024-06-28 15:29:00+05:30
Duration,0 days 06:14:00
Exposure Time [%],92.8
Equity Final [$],99572.8
Equity Peak [$],100028.0
Return [%],-0.4272
Buy & Hold Return [%],-0.165753
Return (Ann.) [%],0.0
Volatility (Ann.) [%],NaN


In [ ]:
stats["_strategy"]

<Strategy Alphabots(base_price=24403.85,delta=0.012)>

In [ ]:
bt.plot()

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p2209', ...)